## Import & 전역변수

In [5]:
import requests
import json
TOT_CNT = 82557
MAX_PER_PAGE = 100
ITER_NUM = TOT_CNT//MAX_PER_PAGE+1
CHECK_POINT = 200

## 영화진흥위원회 API를 사용하여 영화목록 가져오기


In [2]:
URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
for i in range(ITER_NUM):
    PAGE_NUM = i+1
    params = {
        "key" : "74c13a6087207fc051ab22dd0752be12",
        "curPage" : PAGE_NUM,
        "itemPerPage" : MAX_PER_PAGE,
    }
    response = requests.get(URL, params=params)
    # print(response.json())
    with open(f'./json/movie_list_{PAGE_NUM}.json', 'w') as f :
        json.dump(response.json(), f)

## API를 이용해 저장한 데이터 불러오기


In [6]:
TOTAL_DATA_COUNT = 0
for i in range(ITER_NUM) :
    if i % CHECK_POINT == 0 :
        print(f"Now : {i+1}")
    with open(f'./json/movie_list_{i+1}.json', 'r') as f :
        # content = eval(f.readline().strip())
        content = json.load(f)
        movieListResult = content['movieListResult']
        TOTAL_COUNT = movieListResult['totCnt']
        MOVIE_LIST = movieListResult['movieList']
        TOTAL_DATA_COUNT  += len(MOVIE_LIST)

print(f"DB에 저장된 영화의 갯수 : {TOTAL_COUNT}")
print(f"현재 API로 불러 온 영화의 갯수 : {TOTAL_DATA_COUNT}")

Now : 1
Now : 201
Now : 401
Now : 601
Now : 801
DB에 저장된 영화의 갯수 : 82557
현재 API로 불러 온 영화의 갯수 : 82557


## API로 불러온 컬럼 분석

In [14]:
NUM_LEN_ONE_COL = set()
NUM_LEN_OTHER_COL = set()

for i in range(ITER_NUM):
    if i % CHECK_POINT == 0 :
        print(f"Now : {i+1}")
    with open(f'./json/movie_list_{i+1}.json', 'r') as f :
        content = json.load(f)
        movieListResult = content['movieListResult']
        MOVIE_LIST = movieListResult['movieList']
        for MOVIE in MOVIE_LIST :
            for key, val in MOVIE.items() :
                if type(val) != list :
                    # if key == 'genreAlt':
                    #     print(type(val), val)
                    #     break
                    NUM_LEN_ONE_COL.add(key)
                else :
                    NUM_LEN_OTHER_COL.add(key)

print(f"len(ONE_COL) : {len(NUM_LEN_ONE_COL)}, ONE_COL : {NUM_LEN_ONE_COL}")
print(f"len(OTHER_COL) : {len(NUM_LEN_OTHER_COL)}, OTHER_COL : {NUM_LEN_OTHER_COL}")

Now : 1
Now : 201
Now : 401
Now : 601
Now : 801
len(ONE_COL) : 11, ONE_COL : {'movieNmEn', 'prdtYear', 'movieNm', 'repNationNm', 'typeNm', 'nationAlt', 'genreAlt', 'movieCd', 'prdtStatNm', 'repGenreNm', 'openDt'}
len(OTHER_COL) : 2, OTHER_COL : {'companys', 'directors'}


## Table 만들기

In [17]:
result = {}
result_genre = []
for i in range(ITER_NUM):
    if i % CHECK_POINT == 0 :
        print(f"Now : {i+1}")
    with open(f'./json/movie_list_{i+1}.json', 'r') as f :
        content = json.load(f)
        movieListResult = content['movieListResult']
        MOVIE_LIST = movieListResult['movieList']
    for MOIVE in MOVIE_LIST :
        movie_code = None
        for key, val in MOIVE.items():
            if type(val) == list :
                continue
            if key == 'movieCd' :
                result[val] = dict()
                movie_code = val
            try :
                if key == 'genreAlt' :
                    val = val.split(',')
                    for gen in val :
                        result_genre.append({movie_code:gen})
                    # print(val)
                    # raise KeyboardInterrupt
                result[movie_code][key] = val
            except KeyError :
                continue

print(f"Num of movies : {len(result)}")
print(f"len of result_genre : {len(result_genre)}")
print(result_genre[:10])


Now : 1
Now : 201
Now : 401
Now : 601
Now : 801
Num of movies : 82557
len of result_genre : 112660
[{'20218755': '드라마'}, {'20218755': '공포(호러)'}, {'20218755': '미스터리'}, {'20191202': '액션'}, {'20191202': '스릴러'}, {'20218754': '성인물(에로)'}, {'20218769': '성인물(에로)'}, {'20198161': '스릴러'}, {'20218240': '미스터리'}, {'20218240': '공포(호러)'}]


## Pickle, Excel 파일로 저장하기

In [19]:
import pickle
# with open("movie.pickle", 'wb') as bf:
#     pickle.dump(result, bf)

# import pandas as pd
# df = pd.DataFrame.from_dict(result).T
# with pd.ExcelWriter("./excel/movie.xlsx", mode='w') as W:
#     df.to_excel(W)

with open("genre_collection.pickle", 'wb') as bf :
    pickle.dump(result_genre, bf)